In [4]:
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests

def convert_updown_number(text) -> float:
    if text.find('보합') == 0:
        return 0.0

    tokens = text.split()
    if tokens[0] == '하락':
        return float(tokens[1]) * -1
    else :
        return float(tokens[1])

tx = '''하락
				425'''
print(convert_updown_number(tx))

def convert_number(text) -> float:
    negative = 1
    if text[0] == '+':
        text = text[1:]
        negative = 1
    elif text[0] == '-':
        text = text[1:]
        negative = -1

    if text.endswith('%'):
        text = text[:-1]

    text = text.replace(',', '')
    return float(text) * negative

print(convert_number('12,123,123'))

class InvestorVolume:
    def __init__(self, date, personal, foreigner, company, finance, insurance, toosin, bank, etc_fin, gov_fund, etc_comp):
        self.date = date
        self.personal = personal
        self.foreigner = foreigner
        self.company = company
        self.finance = finance
        self.insurance = insurance
        self.toosin = toosin
        self.bank = bank
        self.etc_fin = etc_fin
        self.gov_fund = gov_fund
        self.etc_comp = etc_comp

def crawl_investors(url) -> List[InvestorVolume]:
#url = 'https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=&page=1'
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    sel = dom.select('table.type_1 > tr > td.date2')

    sel_c: List[InvestorVolume] = []

    for trd in sel:
        tr = trd.parent
        sel_c.append(InvestorVolume(
            tr.select_one('td:nth-child(1)').text,
            convert_number(tr.select_one('td:nth-child(2)').text),
            convert_number(tr.select_one('td:nth-child(3)').text),
            convert_number(tr.select_one('td:nth-child(4)').text),
            convert_number(tr.select_one('td:nth-child(5)').text),
            convert_number(tr.select_one('td:nth-child(6)').text),
            convert_number(tr.select_one('td:nth-child(7)').text),
            convert_number(tr.select_one('td:nth-child(8)').text),
            convert_number(tr.select_one('td:nth-child(9)').text),
            convert_number(tr.select_one('td:nth-child(10)').text),
            convert_number(tr.select_one('td:nth-child(11)').text)
        ))
    return sel_c

# 1-> KOSPI, 2-> KOSDAQ, 3->선물
url = 'https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240823&sosok=02&page=1'
# url = 'https://finance.naver.com/sise/investorDealTrendTime.naver?bizdate=20240816&sosok=&page=1'
for a in crawl_investors(url):
    print(a.__dict__)

-425.0
12123123.0
{'date': '24.08.23', 'personal': -203.0, 'foreigner': 728.0, 'company': -202.0, 'finance': 51.0, 'insurance': 32.0, 'toosin': -192.0, 'bank': -8.0, 'etc_fin': -180.0, 'gov_fund': 95.0, 'etc_comp': -323.0}
{'date': '24.08.22', 'personal': 1745.0, 'foreigner': -817.0, 'company': -746.0, 'finance': 35.0, 'insurance': 6.0, 'toosin': -635.0, 'bank': -7.0, 'etc_fin': -149.0, 'gov_fund': 4.0, 'etc_comp': -183.0}
{'date': '24.08.21', 'personal': 3639.0, 'foreigner': -3343.0, 'company': -47.0, 'finance': 276.0, 'insurance': 35.0, 'toosin': -414.0, 'bank': -4.0, 'etc_fin': 4.0, 'gov_fund': 57.0, 'etc_comp': -249.0}
{'date': '24.08.20', 'personal': -1106.0, 'foreigner': 1005.0, 'company': 156.0, 'finance': 218.0, 'insurance': 25.0, 'toosin': -204.0, 'bank': -5.0, 'etc_fin': -54.0, 'gov_fund': 176.0, 'etc_comp': -55.0}
{'date': '24.08.19', 'personal': 1394.0, 'foreigner': -1130.0, 'company': -134.0, 'finance': -59.0, 'insurance': 31.0, 'toosin': -43.0, 'bank': 0.0, 'etc_fin': -54

In [48]:
from datetime import datetime

# dt = '20240804' #to 24.08.04
# today = datetime.strptime(dt, '%Y%m%d')
# today.strftime('%y.%m.%d')

# today = datetime.today()
# print(today.strftime('%Y%m%d'))

def write_investors_volume2db(market_code,
                              target_date,
                              i_personal,
                              i_foreigner,
                              company,
                              finance,
                              insurance,
                              toosin,
                              bank,
                              etc_fin,
                              gov_fund,
                              etc_comp) :
    conn = pymysql.connect(host='horusa', user='root', password='root', db='zio', charset='utf8mb4')
    cur = conn.cursor()

    sql = f"""
    INSERT INTO CRAWL_INVESTOR_DAILY_VOLUME(TYPE_CODE, DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND)  
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
        TYPE_CODE=values(TYPE_CODE),
        DATEON=values(DATEON),
        PERSONAL=values(PERSONAL),
        FOREIGNER=values(FOREIGNER),
        COMPANY=values(COMPANY),
        FINANCE=values(FINANCE),
        INSURANCE=values(INSURANCE),
        TOOSIN=values(TOOSIN),
        BANK=values(BANK),
        ETC_FIN=values(ETC_FIN),
        GOV_FUND=values(GOV_FUND),
        ETC_FUND=values(ETC_FUND)
    """
    cur.execute(sql, (market_code, target_date, i_personal, i_foreigner, company, finance, insurance, toosin, bank, etc_fin, gov_fund, etc_comp))

    conn.commit()
    conn.close()


In [54]:
# processing today time data
from datetime import datetime

url = 'https://finance.naver.com/sise/investorDealTrendTime.naver?bizdate=20240816&sosok=02&page=1'
# for a in crawl_investors(url):
#     print(a.__dict__)
def crawlRealtime2db(targetDt: str = '20240816') :
    dt = datetime.strptime(targetDt, '%Y%m%d')
    dt_converted = dt.strftime('%y.%m.%d')
    print(dt_converted)
    
    lst_crawled = crawl_investors(url)
    if len(lst_crawled) != 0 :
        target_data = lst_crawled[0]
        target_data.date = dt_converted
        # print(target_data.__dict__)
        
        write_investors_volume2db(market_code='02',
                                  target_date=target_data.date,
                                  i_personal=target_data.personal,
                                  i_foreigner=target_data.foreigner,
                                  company=target_data.company,
                                  finance=target_data.finance,
                                  insurance=target_data.insurance,
                                  toosin=target_data.toosin,
                                  bank=target_data.bank,
                                  etc_fin=target_data.etc_fin,
                                  gov_fund=target_data.gov_fund,
                                  etc_comp=target_data.etc_comp
                                  )
        print('updated -> ' + target_data.__dict__.__str__())
    else : 
        print('No data today')
    
crawlRealtime2db(targetDt='20240816')

24.08.16
updated -> {'date': '24.08.16', 'personal': -1344.0, 'foreigner': 1010.0, 'company': 659.0, 'finance': 9.0, 'insurance': 22.0, 'toosin': 427.0, 'bank': -14.0, 'etc_fin': -43.0, 'gov_fund': 258.0, 'etc_comp': -325.0}


20240817


In [27]:
import pymysql

def crawl2db(from_date='20240817', market_code = '02', page = 1):
    target_url = 'https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=%s&sosok=%s&page=%s' % (from_date, market_code, page)
    print(target_url)
    data = crawl_investors(target_url)
    
    for row in data:
        print(row.__dict__)
    
    conn = pymysql.connect(host='horusa', user='root', password='root', db='zio', charset='utf8mb4')
    cur = conn.cursor()
    
    # sql = "INSERT INTO CRAWL_INVESTOR_DAILY_VOLUME(TYPE_CODE, DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s); "

    sql = f"""
    INSERT INTO CRAWL_INVESTOR_DAILY_VOLUME(TYPE_CODE, DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND)  
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
        TYPE_CODE=values(TYPE_CODE),
        DATEON=values(DATEON),
        PERSONAL=values(PERSONAL),
        FOREIGNER=values(FOREIGNER),
        COMPANY=values(COMPANY),
        FINANCE=values(FINANCE),
        INSURANCE=values(INSURANCE),
        TOOSIN=values(TOOSIN),
        BANK=values(BANK),
        ETC_FIN=values(ETC_FIN),
        GOV_FUND=values(GOV_FUND),
        ETC_FUND=values(ETC_FUND)
    """
    
    for row in data:
        try:
            cur.execute(sql, (market_code, row.date, row.personal, row.foreigner, row.company, row.finance, row.insurance, row.toosin, row.bank, row.etc_fin, row.gov_fund, row.etc_comp))
        except Exception as e:
            print('error ---> ' + row.date + ' --> ' + str(e))

    conn.commit()
    conn.close()
    
crawl2db()    

https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240817&sosok=02&page=1
{'date': '24.08.16', 'personal': -1344.0, 'foreigner': 1010.0, 'company': 659.0, 'finance': 9.0, 'insurance': 22.0, 'toosin': 427.0, 'bank': -14.0, 'etc_fin': -43.0, 'gov_fund': 258.0, 'etc_comp': -325.0}
{'date': '24.08.14', 'personal': -807.0, 'foreigner': 828.0, 'company': 8.0, 'finance': -84.0, 'insurance': 22.0, 'toosin': 67.0, 'bank': -2.0, 'etc_fin': -47.0, 'gov_fund': 52.0, 'etc_comp': -29.0}
{'date': '24.08.13', 'personal': 2406.0, 'foreigner': -1742.0, 'company': -689.0, 'finance': 2.0, 'insurance': -29.0, 'toosin': -408.0, 'bank': 0.0, 'etc_fin': -33.0, 'gov_fund': -220.0, 'etc_comp': 25.0}
{'date': '24.08.12', 'personal': 1716.0, 'foreigner': -1550.0, 'company': -157.0, 'finance': -99.0, 'insurance': 20.0, 'toosin': 112.0, 'bank': -1.0, 'etc_fin': -67.0, 'gov_fund': -121.0, 'etc_comp': -10.0}
{'date': '24.08.09', 'personal': -1214.0, 'foreigner': 508.0, 'company': 745.0, 'finance': -2

In [12]:
# 2 to 484
import time
for i in range(1, 484):
    print('-------> turn #{}'.format(i))
    crawl2db(page = i)
    time.sleep(1)



-------> turn #1
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240804&sosok=02&page=1
{'date': '24.08.02', 'personal': 2438.0, 'foreigner': -1505.0, 'company': -897.0, 'finance': 867.0, 'insurance': -170.0, 'toosin': -1189.0, 'bank': -23.0, 'etc_fin': 4.0, 'gov_fund': -386.0, 'etc_comp': -36.0}
{'date': '24.08.01', 'personal': -629.0, 'foreigner': 328.0, 'company': 313.0, 'finance': 203.0, 'insurance': 9.0, 'toosin': 164.0, 'bank': -7.0, 'etc_fin': -4.0, 'gov_fund': -52.0, 'etc_comp': -12.0}
{'date': '24.07.31', 'personal': 479.0, 'foreigner': -604.0, 'company': 109.0, 'finance': 546.0, 'insurance': -49.0, 'toosin': -192.0, 'bank': -2.0, 'etc_fin': -4.0, 'gov_fund': -191.0, 'etc_comp': 16.0}
{'date': '24.07.30', 'personal': 852.0, 'foreigner': -1026.0, 'company': 344.0, 'finance': 30.0, 'insurance': -18.0, 'toosin': 448.0, 'bank': -3.0, 'etc_fin': -5.0, 'gov_fund': -108.0, 'etc_comp': -170.0}
{'date': '24.07.29', 'personal': -1452.0, 'foreigner': 1043.0, 'company'